In [2]:
# Load all the important libraries to create the RandomForestClassifier Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [3]:
#import the data set

Data = pd.read_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Dataset\CKD.csv")

In [4]:
Data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [5]:
#Lets check total number of rows and columns 
Data.shape

(399, 25)

In [6]:
#Check the dataset info 
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             399 non-null    float64
 1   bp              399 non-null    float64
 2   sg              399 non-null    object 
 3   al              399 non-null    float64
 4   su              399 non-null    float64
 5   rbc             399 non-null    object 
 6   pc              399 non-null    object 
 7   pcc             399 non-null    object 
 8   ba              399 non-null    object 
 9   bgr             399 non-null    float64
 10  bu              399 non-null    float64
 11  sc              399 non-null    float64
 12  sod             399 non-null    float64
 13  pot             399 non-null    float64
 14  hrmo            399 non-null    float64
 15  pcv             399 non-null    float64
 16  wc              399 non-null    float64
 17  rc              399 non-null    flo

In [7]:
# Lets check the description of data set
Data.describe()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc
count,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000
mean,51.492308,76.459948,0.899749,0.395990,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
std,16.995379,13.492053,1.314769,1.041155,74.864224,49.336046,5.623758,9.215829,2.823323,2.715753,8.157274,2526.204544,0.841006
min,2.000000,50.000000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,0.000000,0.000000,101.000000,27.000000,0.900000,135.000000,4.000000,10.850000,34.000000,6950.000000,4.500000
50%,54.000000,76.459948,0.000000,0.000000,127.000000,44.000000,1.400000,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
75%,64.000000,80.000000,2.000000,0.000000,150.000000,62.500000,3.077356,141.000000,4.800000,14.600000,44.000000,9400.000000,5.100000
max,90.000000,180.000000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000


In [8]:
# Lets check if the data set has NA values
Data[Data.isna().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [9]:
# Lets check if the data set has Null values
Data[Data.isnull().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [10]:
# Lets check how many classes are there in the target variable "Classification"
Data.classification.value_counts()

yes    249
no     150
Name: classification, dtype: int64

# From above output we got to know that there are 250 records in "Yes" class and 150 records in "No" class. so its called as imbalanced data set

In [11]:
# Lets upsample the "No" records to match with "Yes" records
Data_No = Data[Data["classification"]=="no"]

In [12]:
import random

samples_index = random.sample(range(150),99)

In [13]:
Data_No_final = Data_No.iloc[samples_index]

In [14]:
Data_No_final.reset_index(inplace=True,drop=True)

In [15]:
Data_No_final

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,47.0,60.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,109.0,...,41.0,8300.0,5.2,no,no,no,yes,poor,no,no
1,38.0,60.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,91.0,...,46.0,9100.0,5.8,no,no,no,yes,poor,no,no
2,17.0,60.000000,b,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,yes,poor,no,no
3,35.0,70.000000,b,0.0,0.0,normal,normal,notpresent,notpresent,82.0,...,52.0,9400.0,6.1,no,no,no,yes,poor,no,no
4,43.0,60.000000,b,0.0,0.0,normal,normal,notpresent,notpresent,117.0,...,54.0,7400.0,5.4,no,no,no,yes,poor,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,35.0,76.459948,a,0.0,0.0,normal,normal,notpresent,notpresent,99.0,...,48.0,5000.0,5.2,no,no,no,yes,poor,no,no
95,34.0,60.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,91.0,...,48.0,8600.0,4.9,no,no,no,yes,poor,no,no
96,30.0,80.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,89.0,...,52.0,10200.0,5.0,no,no,no,yes,poor,no,no
97,35.0,60.000000,b,0.0,0.0,normal,normal,notpresent,notpresent,105.0,...,43.0,5800.0,6.2,no,no,no,yes,poor,no,no


In [16]:
Data = Data.append(Data_No_final,ignore_index=True)

C:\Users\Vinoth\AppData\Local\Temp\ipykernel_31936\3974467208.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(Data_No_final,ignore_index=True)


In [17]:
Data.classification.value_counts()

yes    249
no     249
Name: classification, dtype: int64

# From above output we have upsampled the "No" class counts to match with "Yes" counts

In [18]:
# Lets convert the nominal categoty columns to numerical columns using one-hot-encoding via pandas get_dummies method

Data = pd.get_dummies(Data,drop_first=True)

In [19]:
Data.head()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.0,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.0,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1


In [20]:
Data.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [21]:
# Lets seperate the dependent and independent variables
dependent = Data[['classification_yes']]
independent = Data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [22]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [23]:
# Lets spli the data to trai  and test set

X_Train,X_Test,Y_Train,Y_Test = train_test_split(independent,dependent,random_state=0,test_size=30)

In [24]:
# Model Parameters 
Model_Params = {'n_estimators':[i for i in range(1,500,50) ],'learning_rate':[0.0000001,0.000001,0.00001,0.0001,0.001,0.01,0.1,1.0,10,100,1000],'algorithm':['SAMME','SAMME.R'],}

In [25]:
# Lets create a model now using the Gridsearch CV

AdaBoost_Model_Creation = GridSearchCV(AdaBoostClassifier(),Model_Params,n_jobs=-1,scoring={"roc_auc_score":'roc_auc','f1_weighted_score':'f1_weighted'},
                                            refit='roc_auc_score',verbose=2)

In [26]:
AdaBoost_Model_Creation.fit(X_Train,Y_Train)

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                           0.01, 0.1, 1.0, 10, 100, 1000],
                         'n_estimators': [1, 51, 101, 151, 201, 251, 301, 351,
                                          401, 451]},
             refit='roc_auc_score',
             scoring={'f1_weighted_score': 'f1_weighted',
                      'roc_auc_score': 'roc_auc'},
             verbose=2)

In [27]:
# The best parameter that has givem high performance scores
AdaBoost_Model_Creation.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 151}

In [28]:
# The best score for the above parameters
AdaBoost_Model_Creation.best_score_

1.0

# Since there are lot of failures in the gridsearch CV results so the entries will be NA. so we don't need those entries to be saved as it may confuse us in future so we are removing those NA entries and have only the valid entries of grid search CV results saved and used for future reference

In [29]:
# Creating as data frame to save the GridSearch CV results
CV_Output = pd.DataFrame.from_dict(AdaBoost_Model_Creation.cv_results_)

In [30]:
# Check the entries count
CV_Output.shape

(220, 24)

In [31]:
# Final Data frame that has valid entries in it
CV_Output

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_learning_rate,param_n_estimators,params,split0_test_roc_auc_score,split1_test_roc_auc_score,...,std_test_roc_auc_score,rank_test_roc_auc_score,split0_test_f1_weighted_score,split1_test_f1_weighted_score,split2_test_f1_weighted_score,split3_test_f1_weighted_score,split4_test_f1_weighted_score,mean_test_f1_weighted_score,std_test_f1_weighted_score,rank_test_f1_weighted_score
0,0.017556,0.009444,0.011502,0.003894,SAMME,0.0,1,"{'algorithm': 'SAMME', 'learning_rate': 1e-07,...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
1,0.142669,0.007983,0.024441,0.000578,SAMME,0.0,51,"{'algorithm': 'SAMME', 'learning_rate': 1e-07,...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
2,0.286784,0.011821,0.041967,0.003086,SAMME,0.0,101,"{'algorithm': 'SAMME', 'learning_rate': 1e-07,...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
3,0.397899,0.021422,0.049743,0.003428,SAMME,0.0,151,"{'algorithm': 'SAMME', 'learning_rate': 1e-07,...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
4,0.551934,0.022659,0.067615,0.003519,SAMME,0.0,201,"{'algorithm': 'SAMME', 'learning_rate': 1e-07,...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0.014663,0.003337,0.015402,0.004371,SAMME.R,1000,251,"{'algorithm': 'SAMME.R', 'learning_rate': 1000...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
216,0.018387,0.003682,0.014123,0.002384,SAMME.R,1000,301,"{'algorithm': 'SAMME.R', 'learning_rate': 1000...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
217,0.011246,0.003988,0.019128,0.006333,SAMME.R,1000,351,"{'algorithm': 'SAMME.R', 'learning_rate': 1000...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66
218,0.014314,0.003165,0.015947,0.000325,SAMME.R,1000,401,"{'algorithm': 'SAMME.R', 'learning_rate': 1000...",0.904255,0.968085,...,0.036499,104,0.904244,0.968053,0.936141,0.881144,0.97848,0.933612,0.036935,66


In [32]:
# Lets save the grid search CV results to the csv file
CV_Output.to_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Grid Serach CV Results\ABC_GridSerachCV_Results.csv")

In [33]:
# Lets test the model with best parameter that we have got agains the test data and check the performance

Y_Predicted = AdaBoost_Model_Creation.predict(X_Test)

In [34]:
Y_Predicted

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1], dtype=uint8)

In [35]:
# Lets check the confusion matrix for the above predicted against the actual results
confusion_matrix(Y_Predicted,Y_Test)

array([[16,  0],
       [ 0, 14]], dtype=int64)

In [36]:
#lets check the classification test report for the predicted against the actual results
print(classification_report(Y_Predicted,Y_Test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        14

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [37]:
AdaBoost_Model_Creation.predict_proba(X_Test)[:,1]

array([0.35473846, 0.20354926, 0.22407553, 0.21892561, 0.80460598,
       0.21485305, 0.89154636, 0.22902494, 0.87448871, 0.25851492,
       0.84424211, 0.83111238, 0.20354926, 0.96500648, 0.25048592,
       0.95369212, 0.22584415, 0.83780329, 0.21584367, 0.59904248,
       0.23001927, 0.96881997, 0.70002547, 0.68478496, 0.22806928,
       0.24896072, 0.37116875, 0.23979346, 0.9610072 , 0.94964721])

In [38]:
# lets check the roc_auc_score results for the predicted against the actual results
roc_auc_score(Y_Test,AdaBoost_Model_Creation.predict_proba(X_Test)[:,1])

1.0

In [39]:
# Wow we can see our auc score is 100 percent and weighted f1 score is also 93 percent lets save out model
import pickle
pickle.dump(AdaBoost_Model_Creation,open(r'C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\ABC_Final_Model.sav','wb'))

In [40]:
# Lets load and test our model
def Final_model_prod(model,columns,stdscaler='None'):
    query_values=[]
    for col_idx in range(0,len(columns)-1):
        query_values.append(float(input("Please enter valid {}: \n Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' ".format(columns[col_idx]))))
    pred_class=model.predict(stdscaler.transform([query_values]))
    
    if pred_class==0:
        print ("This patient doesn't have Chronic Kidney Disorder")
    else:
        print ("This patient have Chronic Kidney Disorder Please proceed proper medication")

In [41]:
# Load the saved model
Final_model= pickle.load(open(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\ABC_Final_Model.sav",'rb'))

# here the scaler is the varaible that we have used for standardising the independent variables so we should use the same variable here  


In [ ]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)

In [ ]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)